# Exploratory Data Analysis

Due the characterstics of the data in this EDA we'll plot the data using different techinques such as plotting in time domain or in frequency domain.

### Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option("display.max_colwidth", None) # setting the maximum width in characters when displaying pandas column. "None" value means unlimited.

import matplotlib.pyplot as plt  # plotting
from glob import glob     # pathname management
import seaborn as sns # for data visualization

#from src.plot.plot import *

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import interp1d  # interpolating a 1-D function
import matplotlib.mlab as mlab  # some MATLAB commands
from src.load.get_data import *

import librosa
import librosa.display


def visualize_sample(
    df,
    sample_id, 
    signal_names=("LIGO Hanford", "LIGO Livingston", "Virgo")
):
    target = df[df['id'] == sample_id]['target'].values
    sample = get_data(sample_id)
    plt.suptitle(f"Strain data for three observatories from sample: {sample_id} | Target:         {target[0]}")
    for i in range(3):
        sns.lineplot(data=sample[i], color=sns.color_palette()[i])
        plt.subplot(4, 1, i + 1)
        plt.plot(sample[i])
        plt.legend([signal_names[i]], fontsize=12, loc="lower right")
        plt.subplot(4, 1, 4)
        plt.plot(sample[i])
    
    plt.subplot(4, 1, 4)
    plt.legend(signal_names, fontsize=12, loc="lower right")
    plt.suptitle(f"Strain data for three observatories from sample: {sample_id} | Target:         {target[0]}")
    plt.show()

    
# function to plot the amplitude spectral density (ASD) plot
def plot_asd(sample_id,sample_rate,signal_length):
    # Get the data
    sample = get_data(sample_id)
    
    # we convert the data to gwpy's TimeSeries for analysis
    for i in range(sample.shape[0]):
        ts = TimeSeries(sample[i], sample_rate=sample_rate)
        ax = ts.asd(signal_length).plot(figsize=(12, 5)).gca()
        ax.set_xlim(10, 1024);
        ax.set_title(f"ASD plots for sample: {sample_id} from {obs_list[i]}");
        
def plot_asd_mix(sample, sample_rate, NFFT, f_min, f_max):

    Pxx_1, freqs = mlab.psd(sample[0], Fs = sample_rate, NFFT = NFFT)
    Pxx_2, freqs = mlab.psd(sample[1], Fs = sample_rate, NFFT = NFFT)
    Pxx_3, freqs = mlab.psd(sample[2], Fs = sample_rate, NFFT = NFFT)

    psd_1 = interp1d(freqs, Pxx_1)
    psd_2 = interp1d(freqs, Pxx_2)
    psd_3 = interp1d(freqs, Pxx_3)

    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 5))
    ax.loglog(freqs, np.sqrt(Pxx_1),"g",label="Detector 1")
    ax.loglog(freqs, np.sqrt(Pxx_2),"r",label="Detector 2")
    ax.loglog(freqs, np.sqrt(Pxx_3),"b",label="Detector 3")

    ax.set_xlim([f_min, f_max])
    ax.set_xlabel("Frequency (Hz)")
    ax.set_ylabel("Hz^-1/2")
    ax.set_title(f"ASD plots for sample: {sample_id}");
    ax.legend()

    plt.show()

# function to plot the Q-transform spectrogram
def plot_q_transform(sample_id):
    # Get the data
    sample = get_data(sample_id)
    
    # we convert the data to gwpy's TimeSeries for analysis
    for i in range(sample.shape[0]):
        ts = TimeSeries(sample[i], sample_rate=sample_rate)
        ax = ts.q_transform(whiten=True).plot().gca()
        ax.set_xlabel('')
        ax.set_title(f"Spectrogram plots for sample: {sample_id} from {obs_list[i]}")
        ax.grid(False)
        
        ax.set_yscale('log')

# function to plot the Q-transform spectrogram side-by-side
def plot_q_transform_sbs(sample_gw_id, sample_no_gw_id):
    # Get the data
    sample_gw = get_data(sample_gw_id)
    sample_no_gw = get_data(sample_no_gw_id)
    
    for i in range(len(obs_list)):
        # get the timeseries
        ts_gw = TimeSeries(sample_gw[i], sample_rate=sample_rate)
        ts_no_gw = TimeSeries(sample_no_gw[i], sample_rate=sample_rate)
        
        # get the Q-transform
        image_gw = ts_gw.q_transform(whiten=True)
        image_no_gw = ts_no_gw.q_transform(whiten=True)

        plt.figure(figsize=(20, 10))
        plt.subplot(131)
        plt.imshow(image_gw)
        plt.title(f"id: {sample_gw_id} | Target=1")
        plt.grid(False)

        plt.subplot(132)
        plt.imshow(image_no_gw)
        plt.title(f"id: {sample_no_gw_id} | Target=0")
        plt.grid(False)
        
        plt.show()
        
def visualize_sample_spectogram(df,
    sample_id, 
    target,
    signal_names=("LIGO Hanford", "LIGO Livingston", "Virgo")
):
    target = df[df['id'] == sample_id]['target'].values
    sample = get_data(sample_id)
    plt.figure(figsize=(16, 5))
    for i in range(3):
        X = librosa.stft(sample[i] / sample[i].max())
        Xdb = librosa.amplitude_to_db(abs(sample))
        plt.subplot(1, 3, i + 1)
        librosa.display.specshow(Xdb, sr=2048, x_axis="time", y_axis="hz", vmin=-30, vmax=50) 
        plt.colorbar()
        plt.title(signal_names[i], fontsize=14)

    plt.suptitle(f"Spectrogram plots for sample: {sample_id}", fontsize=16)
    plt.show()
    
def visualize_sample_mfcc(df,
    sample_id, 
    sr=2048,
    signal_names=("LIGO Hanford", "LIGO Livingston", "Virgo")
):
    target = df[df['id'] == sample_id]['target'].values
    sample = get_data(sample_id)
    plt.figure(figsize=(16, 5))
    for i in range(3):
        mfccs = librosa.feature.mfcc(sample[i] / sample[i].max(), sr=sr)
        plt.subplot(1, 3, i + 1)
        librosa.display.specshow(mfccs, sr=sr, x_axis="time", vmin=-200, vmax=50, cmap="coolwarm")
        plt.title(signal_names[i], fontsize=14)
        plt.colorbar()

    plt.suptitle(f"Mel Frequency Cepstral Coefficients plots for sample: {sample_id}", fontsize=16)
    plt.show()

### Setup variables

In [ ]:
training_labels=pd.read_csv("data/training_labels.csv")
training_labels.head()

To make things easier let's merge the path of the file into the df with the target

With glob we can get all the files in the train directory

In [ ]:
training_paths = glob("../input/g2net-gravitational-wave-detection/train/*/*/*/*")
print("The total number of files in the training set:", len(training_paths))

In [ ]:
ids = [path.split("/")[-1].split(".")[0] for path in training_paths]
paths_df = pd.DataFrame({"path":training_paths, "id": ids})
train_data = pd.merge(left=training_labels, right=paths_df, on="id")
train_data.head()

In [ ]:
train_data.to_csv("data/data_path.csv")

## Plots

Let's check if the source data target is balanced

In [ ]:
train_data['target'].value_counts()

In [ ]:
sns.countplot(data=train_df, x="target")

As we can see the source data is balanced.

Let's plot the signals

In [ ]:
# draw a random sample from the train data
sample_gw_id = train_data[train_data['target'] == 1].sample(random_state=42)['id'].values[0]

In [ ]:
def plot_1(path,
           a = 1,
           b = 0,
           labels = ('LIGO Hanford', 'LIGO Livingston', 'Virgo')
):
    
    training_files = glob(path)
    data = np.load(training_files[0])
    time = [i/2048 for i in range(len(data[0]))]
    fig, ax = plt.subplots(3,1,figsize=(12,10), sharey= True) 
    data_a = np.load(training_files[a]) 
    for i in range(3):
        #ax[i,a].plot(time, data_a[i]*10**(20), label = labels[i])
    #ax[0, a].set_title('Target: 1')
    plt.suptitle(f"Strain data for three observatories from sample: {labels[i]} | Target: {a}")
    sns.lineplot(data=sample[i], ax=ax[i], color=sns.color_palette()[i])
    ax[i].legend([labels[i]])
    ax[i].set_xlim(0, 4096)
    ax[i].set_xticks(ticks=[0, 2048, 4096]);
    ax[i].set_xticklabels(labels=[0, 1, 2]);

    data_b = np.load(training_files[b]) 
    for i in range(3):
        #ax[i,b].plot(time, data_b[i]*10**(20), label = labels[i])
    #ax[0, b].set_title('Target: 0')
    plt.suptitle(f"Strain data for three observatories from sample: {labels[i]} | Target: {b}")
      sns.lineplot(data=data_b[i]*10**(20), ax=ax[i], color=sns.color_palette()[i])
    ax[i].legend([labels[i]])
    ax[i].set_xlim(0, 4096)
    ax[i].set_xticks(ticks=[0, 2048, 4096]);
    ax[i].set_xticklabels(labels=[0, 1, 2]);

In [ ]:
plot_1(training_paths)

In [ ]:
# plot the sample with gravitational wave signal
visualize_sample(train_data,sample_gw_id)

Descibir que se ve royo :

The three plots above show the strain values sampled for 2s at 2048 Hz for id 882722dba9. Out of the three readings, the two LIGO values are similar in amplitude while the Virgo is smaller. Even though this particular sample has gravitaional wave signal, it is burried deep in the instrument noise.


In [ ]:
# draw another random sample from train without gravitational wave signal
sample_no_gw_id = train_data[train_data['target'] == 0].sample(random_state=42)['id'].values[0]

In [ ]:
# plot the sample without gravitational wave signal
visualize_sample(train_data, sample_no_gw_id)

Similarly, for the sample 05552e5b6a without gravitational wave signal, we cannot visually see any signs. The strain is of the order , which is extremely small and can be affected by many external factors. However, as seen in both the sample plots, the strain data is a combination of many frequencies and analysing the signals in frequency domain, instead of the time domain, might give us better insights.

A Fourier Transform is the most commonly used method in maths and signal processing, to decompose the signals into its constituent discrete frequencies. This spectrum of frequencies can be analyzed based on average, power or energy of the signal to get a spectral density plot. We will follow some of the concepts from this tutorial. As it says, one of the ways to visualize a raw signal in frequency domain is by plotting the amplitude spectral density (ASD).

### Spectral density plots

In [ ]:
# let's define some signal parameters
sample_rate = 2048 # data is provided at 2048 Hz
signal_length = 2 # each signal lasts 2 s
NFFT = 4*fs    # the Nyquist frequency -
f_min = 20.
f_max = fs/2

In [ ]:
# plot ASD for sample w/ GW
plot_asd(sample_gw_id,sample_rate,signal_length)

In [ ]:
plot_asd_mix(sample_gw_id,sample_rate,NFFT,f_min,f_max)

These plots are plotted on a log scale for x-axis, and we see that it ranges from 10 Hz ~ 1000 Hz. Although, these limits are for visualization purposes only, it helps us see some peaks for each observatory. A particular frequency can be peculiar in one measurement but remember that the GW signal has to be detected in all three waves to be confirmed. This data here still seems a bit noisy and as showed in the tutorial, if sampled for longer periods of time (on real data), it can give some valuable insights. However, the data in this competition is simulated and we try to find other ways to visualize it.

Just for the sake of completeness, we also plot the spectral density plots for a sample without GW.

In [ ]:
# plot ASD for sample w/o GW
plot_asd(sample_no_gw_id)

In [ ]:
plot_asd_mix(sample_no_gw_id,sample_rate,NFFT,f_min,f_max)

They do seem to have fewer peaks, specially around 200 Hz, but there is so much variability in this data, that it can be concluded with certainty.